In [38]:
import json
import shutil
import pandas as pd

from pathlib import Path

In [3]:
PATH_TO_DATA = Path('../../data/experiments/aggregate_results').resolve()
expert_f = PATH_TO_DATA.joinpath('12-04-2021/14_04_08')
summary_f = PATH_TO_DATA.joinpath('12-05-2021/10_26_23')
gpt_f = PATH_TO_DATA.joinpath('12-05-2021/13_46_24')
studio_f = PATH_TO_DATA.joinpath('02-02-2022/10_07_59')

In [18]:
def get_code_json(dirname):
    prob = dirname.joinpath('test.json')
    code = dirname.joinpath('all_codes.json')
    with open(prob) as f:
        probs = json.load(f)
    with open(code) as f:
        codes = json.load(f)
    
    probs = list(map(lambda x: x.split('human_generated/')[-1], probs))
    probs = list(map(lambda x: x.split('studio21_generated/')[-1], probs))
    codes = list(map(lambda x: 'def code():'+x, codes.values()))
    result = dict()
    for p, c in zip(probs, codes):
        result[p] = c
        
    return result

In [54]:
expert = get_code_json(expert_f)
summary = get_code_json(summary_f)
gpt = get_code_json(gpt_f)
studio = get_code_json(studio_f)
RESULT_PATH = Path('./results').resolve()
DATA_PATH = Path('../../data/human_generated').resolve()
PWD = Path('.').resolve()

def view_code(fname, out_file=False):
    expert_txt = fname+'/expert.txt'
    summary_txt = fname+'/summary.txt'
    original_txt = fname+'/question.txt'

    exp_str = '\nOriginal Generated Code for expert.txt\n'
    exp_str += expert[original_txt]
    exp_str += '\n\nSummary Generated Code for expert.txt\n'
    exp_str += expert[expert_txt]
    exp_orig_code = expert[original_txt] + '\ncode()'
    exp_sum_code = expert[expert_txt] + '\ncode()'

    sum_str = '\n\nOriginal Generated Code for summary.txt\n'
    sum_str += summary[original_txt]
    sum_str += '\n\nSummary Generated Code for summary.txt\n'
    sum_str += summary[summary_txt]
    sum_orig_code = summary[original_txt] + '\ncode()'
    sum_sum_code = summary[summary_txt] + '\ncode()'

    stu_str = '\n\nOriginal Generated Code for studio21.txt\n'
    stu_str += studio[original_txt]
    stu_str += '\n\nSummary Generated Code for studio21.txt\n'
    stu_str += studio[expert_txt]
    stu_orig_code = studio[original_txt] + '\ncode()'
    stu_sum_code = studio[expert_txt] + '\ncode()'

    out_str = f'Code for problem {fname}\n' + exp_str + sum_str + stu_str

    if out_file:
        new_path = RESULT_PATH.joinpath(fname)
        new_path.mkdir(parents=True, exist_ok=True)
        res_fname = new_path.joinpath('codes.txt')
        with open(res_fname, 'w') as f:
            f.write(out_str)
        res_fname = new_path.joinpath('expert_results.txt')
        with open(res_fname, 'w') as f:
            f.write(exp_str)
        res_fname = new_path.joinpath('summary_results.txt')
        with open(res_fname, 'w') as f:
            f.write(sum_str)
        res_fname = new_path.joinpath('studio21_results.txt')
        with open(res_fname, 'w') as f:
            f.write(stu_str)

        res_fname = new_path.joinpath('expert_original_code.py')
        with open(res_fname, 'w') as f:
            f.write(exp_orig_code)
        res_fname = new_path.joinpath('expert_summary_code.py')
        with open(res_fname, 'w') as f:
            f.write(exp_sum_code)

        res_fname = new_path.joinpath('summary_original_code.py')
        with open(res_fname, 'w') as f:
            f.write(sum_orig_code)
        res_fname = new_path.joinpath('summary_summary_code.py')
        with open(res_fname, 'w') as f:
            f.write(sum_sum_code)

        res_fname = new_path.joinpath('studio21_original_code.py')
        with open(res_fname, 'w') as f:
            f.write(stu_orig_code)
        res_fname = new_path.joinpath('studio21_summary_code.py')
        with open(res_fname, 'w') as f:
            f.write(stu_sum_code)

        test_case_src = DATA_PATH.joinpath(fname, 'input_output.json')
        test_case_dst = new_path.joinpath('input_output.json')
        shutil.copy(test_case_src, test_case_dst)

        script_src = PWD.joinpath('jq_script.sh')
        script_dst = new_path.joinpath('jq_script.sh')
        shutil.copy(script_src, script_dst)

        solutions = DATA_PATH.joinpath(fname, 'solutions.json')
        with open(solutions) as f:
            sols = json.load(f)
        solutions = new_path.joinpath('solution.py')
        with open(solutions, 'w') as f:
            f.write(sols[0])
    else:
        print(out_str)

In [56]:
view_code('train/competition/2033', out_file=True)

In [52]:
view_code('train/competition/2004', out_file=True)

In [58]:
ser = pd.read_csv('sorted_studio_better.csv', header=None)

In [68]:
for _, s in ser.iterrows():
    view_code(s[0], out_file=True)